# **Resolución de la ecuación de Schrödinger unidimensional para un potencial cuadrado. Estudio del coeficiente de transmisión.**
## Problema Voluntario 3
### Física Computacional - 2025

---

**Irene Blanco Ramos**

**Fecha:** 11 de Junio de 2025

---


# Índice

1. [Introducción](#Introduccion)
2. [Resolución de la Ecuación de Schrödinger](#Resolucion-de-la-Ecuacion-de-Schrodinger)
3. [Dependencia del coeficiente de transmisión con los parámetros de la simulación](#Dependencia-del-coeficiente-de-transmision-con-los-parametros-de-la-simulacion)
4. [Cálculo de los valores esperados de observables](#Calculo-de-los-valores-esperados-de-observables)
5. [Potencial multibarrera](#Potencial-multibarrera)
6. [Optimización y paralelización](#Optimizacion-y-paralelizacion)

---

# Introducción 

El objetivo de este informe es estudiar el coeficiente de transmisión de una partícula cuántica que atraviesa una barrera de potencial cuadrado de altura finita. Para ello, se ha realizado una simulación en C que resulve numéricamente la ecuación de Schrodinger unidimensional y se ha calculado el coeficiente de transmisión en diferentes casos imponiendo una función de onda inicial gaussiana para la partícula. 

En sistemas clásicos, una partícula es capaz de atravesar una barrera de energía potencial $V_0$ siempre que su energía cinética $E$ sea mayor que esta altura del potencial. Es caso contrario, la partícula rebotaría cuando $E<V_0$. Es decir, que la probabilidad de que la partícula traspase la barrera de potencial va a ser 0 o 1. 

<p align="center">
  <img src="efectotunel.jpg" alt="Etunel" width="300">
</p>

Sin embargo, en mecánica cuántica se pone de manifiesto el efecto túnel debido a la naturaleza ondulatoria de la materia. Cada partícula lleva asociada una función de onda de forma que existe una probabilidad no nula de que se atraviese la barrera cuando $E<V_0$. La función de onda decae exponencialmente al entrar en la barrera pero, como tiene una anchura finita, la función de onda no se anula al otro lado por lo que una parte es transmitida. Además, cuando la energía de la partícula es $E>V_0$, la partícula tiene suficiente energía como para atravesar la barrera aunque también hay una probabilidad de que sea reflejada. Esto se cuantifica con los coeficientes de reflexión y transmisión, que pueden oscilar entre cero y uno. En el informe nos centraremos en el coeficiente de transmisión $K$.

---

# Resolución de la Ecuación de Schrödinger

Para resolver la ecuación de Schrodinger numéricamente se ha utilizado un método de discretización de la ecuación mediante diferencias finitas. Se crea una malla unidimensional (en la dirección $x$) discretizando tanto el dominio espacial como el temporal. Previamente se ha realizado un reescalamiento de la masa $m=1/2$ de la partícula y la constante de Planck $\hbar=1$ para simplificar las ecuaciones a resolver. 

Se ha dividido el espacio en $N$ puntos con un paso de $h$ que se ha elegido como $h=0.01$ para todas las simulaciones. El tiempo se ha dividido en $T$ pasos que representan el tiempo total de evolución de la función de onda. Para cada instante de tiempo en una posición concreta $x=jh$, el algoritmo resuelve un sitema de matrices tridiagonales para calcular la correspondiente función de onda. Se parte inicialmente de un paqute de ondas gaussiano
$$
\Phi(x, 0) = e^{ik_0x}e^{-(x-x_0)^2/2\sigma^2}
$$

centrado en el punto $x=Nh/4$ y de anchura $\sigma= Nh/16$.
La anchura de la barrera de potencial, centrado en $N/2$, ha sido $N/5$ y su altura depende de la energía de la función de onda incidente $\lambda k_0^2$, donde $\lambda=\frac{E}{V_0}$.

El parámetro $k_0$ es reescalado a $\tilde{k}_0 = k_0 h = 2\pi n_{ciclos}/N$ donde se ha mantenido  $n_{ciclos} = N/16$ en cada simulación. Esta  relación entre $k_0$ y $n_{ciclos}$ refleja el número de oscilaciones completas que la función de onda realiza en la red.

---

# Dependencia del coeficiente de transmisión con los parámetros de la simulación
## Coeficiente de transmisión y probabilidad máxima
Para calcular el coeficiente de transmisión $K$, se ha calculado la probabilidad de encontrar la partícula, que incide desde la izquierda en el potencial, a la derecha de la barrera. Para conseguirlo se ha colocado un detector al final de la barrera de un ancho de $N/5$ tal que la probabilidad de encontrar la partícula en esa región se calcula como:
$$
P_D(n) = \sum_{j=4N/5}^{N} |\Phi_{j,n}|^2.
$$
donde $\Phi_{j,n}$ es la función de onda en la posición $j<N$ en el intante $n<T$. 

Para determinar el tiempo necesario de evolución de la función de onda antes de tomar una medida con el detector, se deja evolucionar primero la función de onda el tiempo $T$ calculando la probabilidad en cada instante $n$. Después, se calcula el instante $n_D$ en el que la probabilidad $P_D$ sea máxima, dejando ahora evolucionar la función de onda $n_D$ pasos de tiempo. Esto se repite para $m$ repeticiones, de manera que en cada repetición se compare $P_D(n_D)$ con un número $p$ aleatorio entre 0 y 1. Cuando $p<P_D(n_D)$ la partícula se habrá detectado a la derecha y se actualiza un contador $m_T$. Si $p>P_D(n_D)$, la partícula no se ha detectado y el contador no suma +1. Finalmete, el coeficiente de transmisión, vendrá dado por el número de medidas detectadas $m_T$ entre el número de medidas totales $m$:
$$
K=\frac{m_T}{m}
$$

## Coeficiente de transmisión y altura del potencial $\lambda$

-   $E < V_0$:

    En este caso, $K = \frac{4E(V_0-E)}{4E(V_0-E)+V_0^2 \sinh^2(k_1 a)}$

-   $E > V_0$:

    En este caso, $K = \frac{4E(E-V_0)}{4E(E-V_0)+V_0^2 \sinh^2(k_1 a)}$

$$
K(\lambda) = \frac{4(1 - \lambda)}{4(1 - \lambda) + \lambda^2 \text{sen}^2 \left(\frac{2\pi}{5} n_{ciclos} \sqrt{1 - \lambda}\right)}
$$
si $\lambda < 1$

Y por:

$$
K(\lambda) = \frac{4(\lambda - 1)}{4(\lambda - 1) + \lambda^2 \text{senh}^2 \left(\frac{2\pi}{5} \sqrt{\lambda - 1}\right)}
$$
si $\lambda > 1$.

<p align="center">
  <img src="Kteorico.png" alt="Kteorico">
</p>

<p align="center">
  <img src="Kexperimental.png" alt="Kexperimental">
</p>

<p align="center">
  <video src="lambda05.mp4" width="400" controls></video>
  <video src="lambda5.mp4" width="400" controls></video>
</p>

## Coeficiente de transmisión y parámetro espacial $N$

<p align="center">
  <img src="KvsNgrafico.png" alt="KvsNgrafico">
</p>

---

# Cálculo de los valores esperados de observables
Para calcular el valor esperado de un observable $A$ se utiliza:
$$
\langle A \rangle = \int \phi^*(x) \hat{A} \phi(x) dx
$$
donde $\hat{A}$ es el operador del observable correspondiente.  
En Física cuática la incertidumbre de una medida procede del origen probabilístico de medir un observable y el principio de indertidumbre de Heisenberg. La incertidumbre de un observable está dada por la desviación de su valor medido con respecto al valor esperado, por lo que se calcula como:
$$
\Delta A = \sqrt{\langle \hat{A}^2 \rangle - (\langle \hat{A} \rangle)^2}
$$
Con esta expresión se han calculado las barras de error de las gráficas que representan la evolución temporal de los valores esperados de la posición y de la energía cinética, que se muestran a continuación. Cada observable se ha representado para dos alturas de potencial diferentes $\lambda=0.3$ y $\lambda=1.0$. 
## Valor esperado de la posición y la energía cinética

Para el valor esperado de la posición, se ha empleado la expresión anterior sustituyendo el operador posición $\hat{x} = x$.



Para calcular el valor esperado de la energía cinética se ha utilizado el operador momento $\hat{p} = -i\hbar \frac{\partial}{\partial x}$, ya que la energía cinética es:
$$
E=\frac{\hat{p}^2}{2m}
$$

<p align="center">
  <img src="graficoxTesperados.png" alt="xTesperados">
</p>

---

## Potencial multibarrera

<p align="center">
  <img src="Kbarrerasgrafico.png" alt="Kbarreras">
</p>

<p align="center">
  <video src="multibarrera1.mp4" width="400" controls></video>
  <video src="multibarrera2.mp4" width="400" controls></video>
</p>